# 5. Inferencia y grados de libertad

Importamos los datos y módulos a usar.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle  

import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
path = '../Data/'

X = np.load(path + 'X.dat', allow_pickle= True)
B = np.load(path + 'B.dat', allow_pickle= True)
Y = np.load(path + 'Y.dat', allow_pickle= True)

In [4]:
result =  sm.load(path + 'ols_results.pickle')

In [5]:
with open("../Data/regressor_columns.pickle", "rb") as fp:   #Pickling
...   regressor_columns = pickle.load(fp)

In [6]:
df = pd.read_csv(path+'Ames-Housing-regressor-columns.csv', delimiter=',')

## 5.1 Calculo de grados de libertad

Los grados de libertad de modelo corresponden a número de coeficientes B estimados, asociados a variables regresoras. Es decir, es calculado como:

$$Df_{Model} = m - 1 = k$$

In [7]:
Df_Model = len(B) - 1

Df_Model

14

In [8]:
result.df_model

14.0

Por su parte, los grados de libertad de los residuos corresponden al número de observaciones n menos el total de coeficientes estimados:

$$Df_{Residuals} = n - m = n - (k + 1)$$

In [9]:
Df_Residuals = len(Y) - len(B)

Df_Residuals

2781

In [10]:
result.df_resid

np.float64(2781.0)

Comparando con los grados de libertad total, que se calcula como n - 1, por la perdida de un grado de libertad por la estimación de $\overline{Y}$:

$$Df_{Total} = n - 1$$

Por lo que se cumple que:

$$Df_{Total} = n - 1 - k + k$$
$$Df_{Total} = n - (k + 1) + k$$
$$Df_{Total} = Df_{Residuals} + Df_{Model}$$

In [11]:
Df_Total = len(Y) - 1

Df_Total

2795

In [12]:
Df_Residuals + Df_Model

2795

## 5.2 Calculo de error estandar

Para calcular el error estándar de cada coeficiente, primero calculamos el error medio cuadrático, usando la expresión:

$$MSE = \sum \frac{(\hat{y} - y)^2}{df_{residuals}}$$

In [13]:
Y_pred = X @ B

MSR = np.sum((Y_pred - Y)**2)/(Df_Residuals)

MSR

np.float64(0.0193693613821067)

Luego, se obtiene la matriz de covarianza de los estimadores calculada como:

$$COV = (X^T X)$$

De esta matriz, nos interesa los valores que están en la diagonal central, los cuales corresponderán a la covarianza de los estimadores, incluyendo $B_0$.

Además, dado que para calcular el error estandar es necesario dividir el error cuadratico medio sobre los valores de la covarianza, es necesario sacar su inversa. De esta manera calculamos el error estandar como:

In [14]:
cov_beta = MSR * np.linalg.inv(X.T @ X)

std_error = np.sqrt(np.diag(cov_beta))

std_error
    

array([4.59975362e-01, 3.40523686e-03, 1.39015484e-02, 8.86817431e-03,
       3.11129778e-05, 1.71438921e-04, 1.06522902e-02, 1.65539486e-02,
       1.87784921e-04, 7.26835675e-03, 8.06223581e-03, 2.05303841e-04,
       7.69283493e-03, 6.24573388e-03, 1.54011420e-02])

## 5.3 Testeo de hipótesis y p-valor

Con el error estándar podemos obtener un estadístico t, que nos ayude a validar la hipotesis de significancia del modelo, donde:

$$H_0: B_i = 0, \forall i > 0$</p> <p style="text-align:center;">$H_1: B_i \ne 0, \forall i > 0$$

Se entiende que si no se puede rechazar la hipotesis nula que el coeficiente asociado a un atributo $X_i$ es igual a 0, entonces ese atributo no tiene una significancia verdadera en el modelo, por lo cual puede ser descartado. El estadistico t asociado será igual a:
$$t_i = \frac{B_i}{se_i}, \forall i > 0$$

Y el p valor será igual a:
$$p_i = 2 * P(t>|t_i|), \forall i > 0$$

Una hipótesis similar se puede realizar para el intercepto $B_0$, donde se evalúa la hipótesis de que el modelo pase por el origen, es decir, $Y=0$ para $X = 0$.

## 5.4 Tabla de coeficientes

Obtenemos la tabla de coeficientes otorgada por statmodels. Comparamos los errores estándares con los calculados y verificamos que efectivamente obtuvimos los mismos resultados:

In [15]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.853
Model:                            OLS   Adj. R-squared:                  0.852
Method:                 Least Squares   F-statistic:                     1154.
Date:                Mon, 10 Nov 2025   Prob (F-statistic):               0.00
Time:                        00:59:52   Log-Likelihood:                 1554.0
No. Observations:                2796   AIC:                            -3078.
Df Residuals:                    2781   BIC:                            -2989.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3401      0.460      0.739      0.460      -0.562       1.242
x1             0.0813      0.003     23.869      0.000       0.075       0.088
x2             0.3533      0.014     25.413      0.000       0.326       0.381
x3             0.0069      0.009      0.773      0.439      -0.011       0.024
x4             0.0002   3.11e-05      7.697      0.000       0.000       0.000
x5             0.0024      0.000     13.793      0.000       0.002       0.003
x6             0.1839      0.011     17.262      0.000       0.163       0.205
x7            -0.0640      0.017     -3.866      0.000      -0.096      -0.032
x8             0.0022      0.000     11.731      0.000       0.002       0.003
x9            -0.0352      0.007     -4.845      0.000      -0.049      -0.021
x10            0.0157      0.008      1.948      0.051      -0.000       0.032
x11           -0.0008      0.000     -4.097      0.000      -0.001      -0.000
x12           -0.0299      0.008     -3.883      0.000      -0.045      -0.015
x13           -0.0705      0.006    -11.286      0.000      -0.083      -0.058
x14           -0.0621      0.015     -4.032      0.000      -0.092      -0.032
==============================================================================
Omnibus:                      256.879   Durbin-Watson:                   1.696
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              693.979
Skew:                          -0.509   Prob(JB):                    2.02e-151
Kurtosis:                       5.219   Cond. No.                     6.04e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.04e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Analizando la tabla, llegamos a las siguientes conclusiones:
* El intercepto obtenido es negativo, con valor estimado de -1.2768, y se rechaza la hipótesis nula que sea 0.
* Los parámetros $B_1, B_2, B_4, B_5, B_6, B_8$ y $B_{10}$ son positivos, con p-valor inferior a 5%, por lo que se rechaza la hipótesis nula que sean 0 y no tengan significancia en el modelo. Que sean negativos implica un incremento en el atributo $X_i$ implica un aumento en el precio de venta.
  
* Los parámetros $B_7, B_9, B_{11}, B_{12}, B_{13}$ y $B_{14}$ son negativos, con p-valor inferior a 5%, por lo que se rechaza la hipótesis nula que sean 0 y no tengan significancia en el modelo. Que sean negativos implica un incremento en el atributo $X_i$ implica una reducción en el precio de venta.
  
* Los parámetros $B_3, B_{10}$ tienen un p-valor superior al 5%, por lo que no se puede rechazar la hipótesis nula de que es igual a 0. Por tanto, podemos asumir que su significancia en el modelo es nula, en comparación a los otros parámetros.

* Algunos coeficientes, como $B_4$, presentan un valor cercano a 0, aunque el p-valor sigue siendo inferior al 5%. Esto puede deberse a que el atributo $X_4$ se mueve en un rango de valores altos, lo que hace que su coeficiente asociado sea bajo, pero sin perder su significancia.

* El p-valor para el intercepto no nos permite rechazar la hipótesis nula de que sea distinto de 0, por ende, se puede asumir que el origen hace parte de la superficie descrita por el modelo.

## 5.5 Analisis de variables significativas

De esta manera, llegamos a la conclusión que las variables más significativas del modelo son:


In [16]:
regressor_columns

['Overall Qual',
 'Gr Liv Area',
 'Garage Cars',
 'Garage Area',
 'Year Built',
 '1st Flr SF',
 'Exter Qual_TA',
 'Year Remod/Add',
 'Full Bath',
 'Foundation_PConc',
 'Garage Yr Blt',
 'Kitchen Qual_TA',
 'Fireplace Qu_Sin categoria',
 'Exter Qual_Gd']

In [17]:
omitted_regressors = [regressor_columns[2], regressor_columns[9]]

regressor_columns_def = [regressor for regressor in regressor_columns if regressor not in omitted_regressors]

regressor_columns_def

['Overall Qual',
 'Gr Liv Area',
 'Garage Area',
 'Year Built',
 '1st Flr SF',
 'Exter Qual_TA',
 'Year Remod/Add',
 'Full Bath',
 'Garage Yr Blt',
 'Kitchen Qual_TA',
 'Fireplace Qu_Sin categoria',
 'Exter Qual_Gd']

### Variables Significantes

A continuación, hacemos un análisis de las variables que encontramos son significantes para el modelo

| **Variable** | **Significado** | **Interpretación** |
|---------------|----------------|--------------------|
| **OverallQual** | *Overall Material and Finish Quality* → Calidad general de los materiales y acabados. | Escala del **1 al 10** que evalúa la calidad general de la construcción y los acabados. Un valor alto indica **mejor calidad** y suele asociarse con precios más altos. |
| **GrLivArea** | *Ground Living Area (sq ft)* → Área habitable sobre el nivel del suelo (en pies²). | Representa el **espacio habitable total por encima del suelo**, sin incluir sótanos ni garajes. Es una de las variables más correlacionadas con `SalePrice`. |
| **GarageArea** | *Garage Area (sq ft)* → Área del garaje (en pies²). | Indica el **tamaño del garaje**, lo que refleja capacidad de estacionamiento o espacio adicional. |
| **YearBuilt** | *Original Construction Year* → Año de construcción original. | Año en que fue construida la vivienda. Las casas más nuevas tienden a tener **mayor valor**, aunque también depende del mantenimiento y remodelaciones. |
| **1stFlrSF** | *First Floor Square Feet* → Superficie del primer piso (en pies²). | Área del **primer nivel** de la vivienda. Puede correlacionarse con `GrLivArea`, pero se refiere específicamente al piso inferior. |
| **YearRemod/Add** | *Remodel Year* → Año de remodelación o ampliación. | Año en que la vivienda fue **remodelada o ampliada**. Si coincide con `YearBuilt`, significa que **no ha sido remodelada**. |
| **FullBath** | *Full Bathrooms Above Grade* → Número de baños completos sobre el nivel del suelo. | Cuenta los baños con ducha o bañera ubicados **en niveles habitables**, lo que impacta en la comodidad y el valor. |
| **GarageYrBlt** | *Year Garage was Built* → Año de construcción del garaje. | Indica cuándo se construyó el garaje. Si es nulo, la vivienda **no tiene garaje**. También refleja la antigüedad de la infraestructura. |

---





### Variables categóricas (dummies de calidad)


| **Variable** | **Significado** | **Interpretación del valor 1** |
|---------------|----------------|-------------------------------|
| **ExterQual_TA** | *Exterior Quality = Typical/Average* → Calidad exterior promedio. | La vivienda tiene un **acabado exterior de calidad típica o media**, funcional pero sin lujos. |
| **ExterQual_Gd** | *Exterior Quality = Good* → Calidad exterior buena. | La vivienda tiene un **acabado exterior de buena calidad**, mejor que el promedio y asociado a un **mayor valor de mercado**. |
| **KitchenQual_TA** | *Kitchen Quality = Typical/Average* → Calidad promedio de la cocina. | La cocina tiene materiales y acabados **de nivel medio**, ni lujosos ni deficientes. |
| **FireplaceQu_Sin categoria** | *Fireplace Quality = Sin categoría / Nulo* | Indica que la **calidad de la chimenea no fue registrada** o que la casa **no tiene chimenea**. Esta categoría fue creada durante la limpieza de datos para representar los valores faltantes. |

---

### Variables descartadas

In [18]:
omitted_regressors

['Garage Cars', 'Foundation_PConc']

---

| **Variable** | **Significado** | **Tipo** | **Interpretación** |
|---------------|----------------|-----------|--------------------|
| **GarageCars** | *Garage Capacity (number of cars)* → Capacidad del garaje medida en número de automóviles. | Numérica | Indica cuántos **vehículos caben en el garaje**. Es un indicador directo del tamaño y funcionalidad del garaje. Valores mayores sugieren **viviendas más amplias y de mayor valor**. Muy correlacionada con `GarageArea` y `SalePrice`. |
| **Foundation_PConc** | *Foundation = PConc (Poured Concrete)* → Tipo de **cimiento de concreto vertido** (losas o muros moldeados en sitio). | Categórica (dummy) | Toma valor **1 si la vivienda tiene cimientos de concreto vertido**, y 0 en caso contrario. Este tipo de cimiento se asocia con **mayor durabilidad, aislamiento térmico y resistencia estructural**, características de **construcciones modernas o de alta calidad**. |

---


De esta manera, calculamos nuevamente nuestro modelo, usando únicamente las variables significantes:

In [19]:
X = df[regressor_columns_def].to_numpy()

## inserción de 1
X_1 = np.insert(X, obj=0, values=1, axis=1)

X_1

array([[1.        , 6.        , 7.41276402, ..., 1.        , 0.        ,
        0.        ],
       [1.        , 5.        , 6.79905586, ..., 1.        , 1.        ,
        0.        ],
       [1.        , 6.        , 7.19293422, ..., 0.        , 1.        ,
        0.        ],
       ...,
       [1.        , 5.        , 6.87832647, ..., 1.        , 1.        ,
        0.        ],
       [1.        , 5.        , 7.23705903, ..., 1.        , 0.        ,
        0.        ],
       [1.        , 7.        , 7.60140233, ..., 1.        , 0.        ,
        0.        ]], shape=(2796, 13))

In [20]:
X_1.shape

(2796, 13)

In [21]:
# Se recalcula el modelo con OLS
model = OLS(Y,X_1)
result = model.fit()

result.params

array([ 3.63760834e-04,  8.19452409e-02,  3.56525529e-01,  2.57807586e-04,
        2.44827723e-03,  1.80751184e-01, -6.51825264e-02,  2.23976855e-03,
       -3.40143085e-02, -7.87580514e-04, -3.18505242e-02, -7.05809641e-02,
       -6.05993824e-02])

In [22]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.853
Model:                            OLS   Adj. R-squared:                  0.852
Method:                 Least Squares   F-statistic:                     1345.
Date:                Mon, 10 Nov 2025   Prob (F-statistic):               0.00
Time:                        00:59:52   Log-Likelihood:                 1551.6
No. Observations:                2796   AIC:                            -3077.
Df Residuals:                    2783   BIC:                            -3000.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0004      0.430      0.001      0.999      -0.843       0.844
x1             0.0819      0.003     24.172      0.000       0.075       0.089
x2             0.3565      0.014     25.786      0.000       0.329       0.384
x3             0.0003   1.77e-05     14.526      0.000       0.000       0.000
x4             0.0024      0.000     14.689      0.000       0.002       0.003
x5             0.1808      0.011     17.125      0.000       0.160       0.201
x6            -0.0652      0.017     -3.939      0.000      -0.098      -0.033
x7             0.0022      0.000     11.971      0.000       0.002       0.003
x8            -0.0340      0.007     -4.712      0.000      -0.048      -0.020
x9            -0.0008      0.000     -3.913      0.000      -0.001      -0.000
x10           -0.0319      0.008     -4.180      0.000      -0.047      -0.017
x11           -0.0706      0.006    -11.378      0.000      -0.083      -0.058
x12           -0.0606      0.015     -3.937      0.000      -0.091      -0.030
==============================================================================
Omnibus:                      262.042   Durbin-Watson:                   1.697
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              718.139
Skew:                          -0.514   Prob(JB):                    1.14e-156
Kurtosis:                       5.260   Cond. No.                     5.65e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.65e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

---
Tras la obtención de nuestro modelo, procedemos a guardarlo para uso futuro.

In [23]:
X_1.dump('../Data/X_def.dat')
result.params.dump('../Data/B_def.dat')
Y.dump('../Data/Y_def.dat')

In [24]:
result.save("../Data/ols_def_results.pickle")

In [25]:
with open("../Data/regressor_columns_def.pickle", "wb") as fp:   #Pickling
...   pickle.dump(regressor_columns_def, fp)